# Environment Settings

In [2]:
import torch
print(torch.version.cuda)


12.4


In [ ]:
import torch

if torch.cuda.is_available():
    gpu_id = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(gpu_id)
    gpu_capability = torch.cuda.get_device_capability(gpu_id)
    total_memory = torch.cuda.get_device_properties(gpu_id).total_memory
    
    print(f"Total number of GPU: {torch.cuda.device_count()}")  
    print(f"Total GPU memory: {total_memory / 1e9} GB")
    print(f"GPU ID: {gpu_id}")
    print(f"GPU Name: {gpu_name}")
    print(f"GPU Compute Capability: {gpu_capability}")
else:
    print("No GPU is available.")

Total number of GPU: 4
Total GPU memory: 51.033931776 GB
GPU ID: 0
GPU Name: NVIDIA RTX A6000
GPU Compute Capability: (8, 6)


In [ ]:
import torch


torch.cuda.set_device(3)


print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


Current Device: 3
Device Name: NVIDIA RTX A6000


device(type='cuda')

In [5]:
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [7]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv('../dataset/exchange_rate.csv', index_col='date', parse_dates=True)

In [9]:
data.head()

,0,1,2,3,4,5,6,OT
date,,,,,,,,
1990-01-01,0.7855,1.6110,0.861698,0.634196,0.211242,0.006838,0.525486,0.5930
1990-01-02,0.7818,1.6100,0.861104,0.633513,0.211242,0.006863,0.523972,0.5940
1990-01-03,0.7867,1.6293,0.861030,0.648508,0.211242,0.006975,0.526316,0.5973
1990-01-04,0.7860,1.6370,0.862069,0.650618,0.211242,0.006953,0.523834,0.5970
1990-01-05,0.7849,1.6530,0.861995,0.656254,0.211242,0.006940,0.527426,0.5985


In [10]:
print(data.dtypes)
print('shape: ', data.shape)

0     float64
1     float64
2     float64
3     float64
4     float64
5     float64
6     float64
OT    float64
dtype: object
shape:  (7588, 8)


In [11]:
target = data['OT']
features = data.drop(['OT'],axis=1)

In [12]:
features['DATE'] = features.index.strftime('%Y%m%d%H').astype(int)

In [13]:
target.index

DatetimeIndex(['1990-01-01', '1990-01-02', '1990-01-03', '1990-01-04',
               '1990-01-05', '1990-01-06', '1990-01-07', '1990-01-08',
               '1990-01-09', '1990-01-10',
               ...
               '2010-10-01', '2010-10-02', '2010-10-03', '2010-10-04',
               '2010-10-05', '2010-10-06', '2010-10-07', '2010-10-08',
               '2010-10-09', '2010-10-10'],
              dtype='datetime64[ns]', name='date', length=7588, freq=None)

In [ ]:
import torch
import numpy as np
import random

def set_seed(seed=42):
    random.seed(seed)                           
    np.random.seed(seed)                        
    torch.manual_seed(seed)                     
    torch.cuda.manual_seed(seed)                
    torch.cuda.manual_seed_all(seed)            
    torch.backends.cudnn.deterministic = True   
    torch.backends.cudnn.benchmark = False      

In [15]:
set_seed(24) 

# Model Modification for DLinear (Decomposition-wise Look-ahead Augmentation)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class Model(nn.Module):
    """
    Decomposition-Linear
    """
    def __init__(self, configs):
        super(Model, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len
        self.individual = configs.individual
        self.channels = configs.enc_in
        self.attach_to_trend = configs.attach_to_trend
        self.attach_to_seasonal = configs.attach_to_seasonal
        

        
        kernel_size = 25
        self.decompsition = series_decomp(kernel_size)
        

        
        self.seq_len_trend = self.seq_len + (self.pred_len // 3) if self.attach_to_trend else self.seq_len
        self.seq_len_seasonal = self.seq_len + (self.pred_len // 3) if self.attach_to_seasonal else self.seq_len

        
        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
                self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))

                
                
                
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
            
            
            
            

    def forward(self, x, ground_truth=None):
        
        seasonal_init, trend_init = self.decompsition(x)
        seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
        if ground_truth is not None:
            kernel_size = 25
            self.decompsition = series_decomp(kernel_size)
            ground_truth_seasonal, ground_truth_trend = self.decompsition(ground_truth)

            
            if self.seq_len_seasonal > self.seq_len:
                ground_truth_seasonal=ground_truth_seasonal.permute(0, 2, 1)
                
                seasonal_init = torch.cat([seasonal_init, ground_truth_seasonal], dim=2)
            if self.seq_len_trend > self.seq_len:
                ground_truth_trend=ground_truth_trend.permute(0, 2, 1)
                
                trend_init = torch.cat([trend_init, ground_truth_trend], dim=2)
        
        
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
                trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)

        x = seasonal_output + trend_output
        return x.permute(0,2,1) 

# Split the Data

In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


target = data['OT'].values.reshape(-1, 1)  


seq_len  = 336
pred_len = 192


total_len  = len(target)
num_train  = int(total_len * 0.7)
num_test   = int(total_len * 0.2)
num_val    = total_len - num_train - num_test

border1s = [
    0,
    num_train - seq_len,
    total_len - num_test - seq_len
]
border2s = [
    num_train,
    num_train + num_val,
    total_len
]


train = target[border1s[0]:border2s[0]]
val   = target[border1s[1]:border2s[1]]
test  = target[border1s[2]:border2s[2]]


scaler = StandardScaler()
train = scaler.fit_transform(train)
val   = scaler.transform(val)
test  = scaler.transform(test)


def create_inout_sequences_univariate(data, seq_len, pred_len):
    seqs = []
    for i in range(len(data) - seq_len - pred_len + 1):
        seq_x = data[i : i + seq_len]
        seq_y = data[i + seq_len : i + seq_len + pred_len]
        seqs.append((seq_x, seq_y))
    return seqs

train_data = create_inout_sequences_univariate(train, seq_len, pred_len)
val_data   = create_inout_sequences_univariate(val,   seq_len, pred_len)
test_data  = create_inout_sequences_univariate(test,  seq_len, pred_len)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_sequences = torch.tensor([x[0] for x in train_data]).float().to(device)
train_labels    = torch.tensor([x[1] for x in train_data]).float().to(device)

val_sequences   = torch.tensor([x[0] for x in val_data]).float().to(device)
val_labels      = torch.tensor([x[1] for x in val_data]).float().to(device)

test_sequences  = torch.tensor([x[0] for x in test_data]).float().to(device)
test_labels     = torch.tensor([x[1] for x in test_data]).float().to(device)


print("Train Sequences:", train_sequences.shape)
print("Train Labels:   ", train_labels.shape)
print("Val Sequences:  ", val_sequences.shape)
print("Val Labels:     ", val_labels.shape)
print("Test Sequences: ", test_sequences.shape)
print("Test Labels:    ", test_labels.shape)


Train Sequences: torch.Size([4784, 336, 1])
Train Labels:    torch.Size([4784, 192, 1])
Val Sequences:   torch.Size([569, 336, 1])
Val Labels:      torch.Size([569, 192, 1])
Test Sequences:  torch.Size([1326, 336, 1])
Test Labels:     torch.Size([1326, 192, 1])


In [ ]:

batch_size = 8


train_dataset = TensorDataset(train_sequences, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


val_dataset = TensorDataset(val_sequences, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


test_dataset = TensorDataset(test_sequences, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# 1st Model Training (Early Stopping)

In [19]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


In [ ]:
import torch.optim as optim
import os


seq_len = 336
pred_len = 192  

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len = seq_len
        self.pred_len = pred_len  
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.0005, lradj='type1',  patience=3, save_path="./model_ER", attach_to_trend=False , attach_to_seasonal=False) 


model = Model(configs).to(device)
loss_function = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)


class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), os.path.join(path, 'checkpoint_ES.pth'))
        self.val_loss_min = val_loss




def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')



def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    early_stopping = EarlyStopping(patience=args.patience, verbose=True)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        
        
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            
            output = model(batch_sequences)
            
            
            
            output = output[:, -pred_len:, :]  
            batch_labels = batch_labels[:, -pred_len:, :]  
            
            
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  
            
            loss.backward()  
            optimizer.step()  

            total_loss += loss.item()

        
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')
        
        
        early_stopping(val_loss, model, args.save_path)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break


train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=10, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.001
Epoch 0 | Train Loss: 0.30643818823365265 | Val Loss: 0.2509513593589266
Validation loss decreased (inf --> 0.250951).  Saving model ...
Updating learning rate to 0.0005
Epoch 1 | Train Loss: 0.2605004128205736 | Val Loss: 0.24704661851541865
Validation loss decreased (0.250951 --> 0.247047).  Saving model ...
Updating learning rate to 0.00025
Epoch 2 | Train Loss: 0.24527559002057764 | Val Loss: 0.4012862617770831
EarlyStopping counter: 1 out of 3
Updating learning rate to 0.000125
Epoch 3 | Train Loss: 0.24017693611939614 | Val Loss: 0.39056946782188284
EarlyStopping counter: 2 out of 3
Updating learning rate to 6.25e-05
Epoch 4 | Train Loss: 0.23908386647975366 | Val Loss: 0.35264115045881933
EarlyStopping counter: 3 out of 3
Early stopping triggered. Stopping training.


# 1st Model Training (1 Epoch)

In [ ]:
import torch.optim as optim
import os



seq_len = 336
pred_len = 192  

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual, learning_rate, lradj, patience, save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len = seq_len
        self.pred_len = pred_len  
        self.enc_in = enc_in
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal

configs = Configs(seq_len=seq_len, pred_len=pred_len, enc_in=1, individual=False,
                   learning_rate=0.0005, lradj='type1',  patience=3, save_path="./model_ER", attach_to_trend=False , attach_to_seasonal=False) 


model = Model(configs).to(device)
loss_function = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)

if not os.path.exists(configs.save_path):
    os.makedirs(configs.save_path)


def adjust_learning_rate(optimizer, epoch, args):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == '3':
        lr_adjust = {epoch: args.learning_rate if epoch < 10 else args.learning_rate*0.1}
    elif args.lradj == '4':
        lr_adjust = {epoch: args.learning_rate if epoch < 15 else args.learning_rate*0.1}
    elif args.lradj == '5':
        lr_adjust = {epoch: args.learning_rate if epoch < 25 else args.learning_rate*0.1}
    elif args.lradj == '6':
        lr_adjust = {epoch: args.learning_rate if epoch < 5 else args.learning_rate*0.1}  

    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f'Updating learning rate to {lr}')



def train_model(model, dataloader, val_dataloader, optimizer, loss_function, epochs, pred_len, args):
    
    for epoch in range(epochs):

        if epoch == 1:
            torch.save(model.state_dict(), os.path.join(args.save_path, 'checkpoint_1.pth'))
            print("Model saved at epoch 1")

        total_loss = 0
        model.train()
        
        
        adjust_learning_rate(optimizer, epoch, args)
        
        for batch_sequences, batch_labels in dataloader:
            optimizer.zero_grad()

            
            output = model(batch_sequences)
            
            
            
            output = output[:, -pred_len:, :]  
            batch_labels = batch_labels[:, -pred_len:, :]  
            
            
            assert output.shape == batch_labels.shape, f"Output shape {output.shape} and batch_labels shape {batch_labels.shape} must match"
            
            loss = loss_function(output, batch_labels)  
            
            loss.backward()  
            optimizer.step()  

            total_loss += loss.item()

        
        val_loss = 0
        model.eval()
        with torch.no_grad():
            for val_batch_sequences, val_batch_labels in val_dataloader:
                val_output = model(val_batch_sequences)
                val_output = val_output[:, -args.pred_len:, :]
                val_batch_labels = val_batch_labels[:, -args.pred_len:, :]
                
                val_loss += loss_function(val_output, val_batch_labels).item()
        
        val_loss /= len(val_dataloader)

        
        print(f'Epoch {epoch} | Train Loss: {total_loss / len(dataloader)} | Val Loss: {val_loss}')

    torch.save(model.state_dict(), os.path.join(args.save_path, f'checkpoint_{epochs}.pth'))
    print(f"Model saved at epoch {epochs}")
        
        
        
        
        
        


train_model(model, train_dataloader, val_dataloader, optimizer, loss_function, epochs=1, pred_len=configs.pred_len, args=configs)




Updating learning rate to 0.001
Epoch 0 | Train Loss: 0.30927395382055073 | Val Loss: 0.3527267710823152
Model saved at epoch 1


# Perform Inference and Form Predicted Value (PV) Sequences

In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


checkpoint_epochs = ['ES', 1]


test_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in test_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    test_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    test_1st_results[ckpt_epoch] = {
        f'mse_test_E{ckpt_epoch}': mse,
        f'mae_test_E{ckpt_epoch}': mae,
        f'test_pv_E{ckpt_epoch}': test_pv
    }






[Predicting with model from epoch ES]
Epoch ES | MSE: 0.3079990744590759 | MAE: 0.4067384600639343

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.27100130915641785 | MAE: 0.42349910736083984


In [23]:
test_1st_results.keys()

dict_keys(['ES', 1])

In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


checkpoint_epochs = ['ES', 1]


val_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in val_dataloader:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    val_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    val_1st_results[ckpt_epoch] = {
        f'mse_val_E{ckpt_epoch}': mse,
        f'mae_val_E{ckpt_epoch}': mae,
        f'val_pv_E{ckpt_epoch}': val_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.2432703822851181 | MAE: 0.3856322765350342

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.35618194937705994 | MAE: 0.48135992884635925


In [ ]:
import torch.optim as optim
import os
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch


train_dataloader_infer = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)


checkpoint_epochs = ['ES', 1]


train_1st_results = {}

for ckpt_epoch in checkpoint_epochs:
    print(f'\n[Predicting with model from epoch {ckpt_epoch}]')

    
    model_path = os.path.join(configs.save_path, f'checkpoint_{ckpt_epoch}.pth')
    
    model = Model(configs).to(device)
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    predictions = []
    actuals = []

    with torch.no_grad():
        for batch_sequence, batch_label in train_dataloader_infer:
            batch_sequence = batch_sequence.float().to(device)
            batch_label = batch_label.float().to(device)

            output = model(batch_sequence)
            output = output[:, -configs.pred_len:, :]
            batch_label = batch_label[:, -configs.pred_len:, :]

            predictions.append(output.detach().cpu().numpy())
            actuals.append(batch_label.detach().cpu().numpy())

    
    predictions = np.concatenate(predictions, axis=0)  
    actuals = np.concatenate(actuals, axis=0)

    
    predictions_flatten = predictions.flatten()
    actuals_flatten = actuals.flatten()

    
    mse = mean_squared_error(actuals_flatten, predictions_flatten)
    mae = mean_absolute_error(actuals_flatten, predictions_flatten)

    print(f'Epoch {ckpt_epoch} | MSE: {mse} | MAE: {mae}')

    
    num_samples = len(predictions_flatten) // configs.pred_len
    train_pv = torch.tensor(
        predictions_flatten.reshape(num_samples, configs.pred_len, 1),
        dtype=torch.float32
    )

    
    train_1st_results[ckpt_epoch] = {
        f'mse_train_E{ckpt_epoch}': mse,
        f'mae_train_E{ckpt_epoch}': mae,
        f'train_pv_E{ckpt_epoch}': train_pv
    }




[Predicting with model from epoch ES]
Epoch ES | MSE: 0.29796698689460754 | MAE: 0.37846583127975464

[Predicting with model from epoch 1]
Epoch 1 | MSE: 0.27174481749534607 | MAE: 0.3623902201652527


In [26]:
print("train_sequences shape: ",train_sequences.shape)
print("val_sequences shape: ",val_sequences.shape)
print("test_sequences shape: ",test_sequences.shape)

print('---------------------------------------------')

print("train_pv shape: ",train_pv.shape)
print("val_pv shape: ",val_pv.shape)
print("test_pv shape: ",test_pv.shape)


print('---------------------------------------------')

print("train_labels shape: ",train_labels.shape)
print("val_labels shape: ",val_labels.shape)
print("test_labels shape: ",test_labels.shape)


train_sequences shape:  torch.Size([4784, 336, 1])
val_sequences shape:  torch.Size([569, 336, 1])
test_sequences shape:  torch.Size([1326, 336, 1])
---------------------------------------------
train_pv shape:  torch.Size([4784, 192, 1])
val_pv shape:  torch.Size([569, 192, 1])
test_pv shape:  torch.Size([1326, 192, 1])
---------------------------------------------
train_labels shape:  torch.Size([4784, 192, 1])
val_labels shape:  torch.Size([569, 192, 1])
test_labels shape:  torch.Size([1326, 192, 1])


# Segment Generation Function (sliding window-based)

In [ ]:
def create_sliding_segments(predictions, labels, segment_length, stride=1):
    """
    시퀀스에 슬라이딩 윈도우를 stride 간격으로 적용하여
    (B, num_segments, segment_length, C) 형태로 반환합니다.

    Args:
        predictions: Tensor of shape (B, T, C)
        labels: Tensor of shape (B, T, C)
        segment_length: int, 각 세그먼트의 길이 (예: pred_len // 3)
        stride: int, 슬라이딩 윈도우 stride 간격

    Returns:
        pred_segments: (B, num_segments, segment_length, C)
        label_segments: (B, num_segments, segment_length, C)
    """
    B, T, C = predictions.shape
    num_segments = (T - segment_length) // stride + 1

    pred_segments = []
    label_segments = []

    for i in range(0, T - segment_length + 1, stride):
        pred_seg = predictions[:, i:i+segment_length, :]  
        label_seg = labels[:, i:i+segment_length, :]      
        pred_segments.append(pred_seg.unsqueeze(1))       
        label_segments.append(label_seg.unsqueeze(1))

    pred_segments = torch.cat(pred_segments, dim=1)       
    label_segments = torch.cat(label_segments, dim=1)     

    return pred_segments, label_segments

# Segmentation for PV Sequence

In [ ]:
configs.pred_len 

192

In [ ]:
segment_len = configs.pred_len // 3
stride = 2

train_segments_by_epoch = {}
val_segments_by_epoch = {}
test_segments_by_epoch = {}

for epoch in checkpoint_epochs:
    train_pv = train_1st_results[epoch][f'train_pv_E{epoch}']  
    val_pv = val_1st_results[epoch][f'val_pv_E{epoch}']        
    test_pv = test_1st_results[epoch][f'test_pv_E{epoch}']     

    
    train_pred_segments, train_label_segments = create_sliding_segments(train_pv, train_labels, segment_len,stride)
    val_pred_segments, val_label_segments = create_sliding_segments(val_pv, val_labels, segment_len, stride)
    test_pred_segments, test_label_segments = create_sliding_segments(test_pv, test_labels, segment_len, stride)

    train_segments_by_epoch[epoch] = {
        f'train_pred_segments_E{epoch}': train_pred_segments,  
        f'train_label_segments_E{epoch}': train_label_segments
    }

    val_segments_by_epoch[epoch] = {
        f'val_pred_segments_E{epoch}': val_pred_segments,
        f'val_label_segments_E{epoch}': val_label_segments
    }

    test_segments_by_epoch[epoch] = {
        f'test_pred_segments_E{epoch}': test_pred_segments,
        f'test_label_segments_E{epoch}': test_label_segments
    }

    
    print(f"[Epoch {epoch}] Train seg shape: {train_pred_segments.shape}, Val: {val_pred_segments.shape}, Test: {test_pred_segments.shape}")


[Epoch ES] Train seg shape: torch.Size([4784, 65, 64, 1]), Val: torch.Size([569, 65, 64, 1]), Test: torch.Size([1326, 65, 64, 1])
[Epoch 1] Train seg shape: torch.Size([4784, 65, 64, 1]), Val: torch.Size([569, 65, 64, 1]), Test: torch.Size([1326, 65, 64, 1])


## Save 1st Performance Metrics, PV, and Segment

In [ ]:
save_dir = './1st_results_ER'  
os.makedirs(save_dir, exist_ok=True)


torch.save(train_1st_results, os.path.join(save_dir, 'train_1st_results.pt'))
torch.save(val_1st_results, os.path.join(save_dir, 'val_1st_results.pt'))
torch.save(test_1st_results, os.path.join(save_dir, 'test_1st_results.pt'))


torch.save(train_segments_by_epoch, os.path.join(save_dir, 'train_segments_by_epoch.pt'))
torch.save(val_segments_by_epoch, os.path.join(save_dir, 'val_segments_by_epoch.pt'))
torch.save(test_segments_by_epoch, os.path.join(save_dir, 'test_segments_by_epoch.pt'))

print("✅ All dictionaries saved successfully in ./saved_results")


✅ All dictionaries saved successfully in ./saved_results


# 2nd Model Training

In [31]:
train_segments_by_epoch = torch.load('./1st_results_ER/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ER/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ER/test_segments_by_epoch.pt', weights_only=False)

In [32]:

print(train_segments_by_epoch.keys())
print(train_segments_by_epoch['ES'].keys())
print(train_segments_by_epoch[1].keys())

dict_keys(['ES', 1])
dict_keys(['train_pred_segments_EES', 'train_label_segments_EES'])
dict_keys(['train_pred_segments_E1', 'train_label_segments_E1'])


In [ ]:

train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']



## ES->1E

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path
import time


epo = 'ES'

class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal



base_cfg = Configs(
    seq_len          = 336 + pred_len//3,
    pred_len         = 192,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ER_E{epo}_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()


for seg in range(len(train_pred_segments_ES[0])):
    print(f"\n=========== Segment {seg} ===========")
    start_time = time.time()

    
    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    
    tr_pv = train_pred_segments_ES[:, seg, :, :]   
    va_pv = val_pred_segments_ES[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=8, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=8, shuffle=False)

    
    model = Model(base_cfg).to(device)
    C     = model.channels                                    

    opts = []
    for c in range(C):
        params = list(model.Linear_Seasonal[c].parameters()) + \
                 list(model.Linear_Trend[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            

    
    for epoch in range(1, 2):
        if not active: break          

        
        model.train()
        for c in active:                              
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False  

        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                
                
                num_batches = len(tr_dl)

                
                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True


            out = model(xb, pv)                       

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()
        
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    
    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()






=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.1318 MB
  yb per batch = 0.0059 MB, total = 3.5039 MB
  pv per batch = 0.0020 MB, total = 1.1680 MB


[Segment 0] Training Time = 3.62 sec
[Segment 0] GPU Memory Start = 460.85 MB
[Segment 0] GPU Memory End   = 355.71 MB
[Segment 0] GPU Peak Memory  = 460.85 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.1318 MB
  yb per batch = 0.0059 MB, total = 3.5039 MB
  pv per batch = 0.0020 MB, total = 1.1680 MB
[Segment 1] Training Time = 3.55 sec
[Segment 1] GPU Memory Start = 355.71 MB
[Segment 1] GPU Memory End   = 356.30 MB
[Segment 1] GPU Peak Memory  = 357.35 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.1318 MB
  yb per batch = 0.0059 MB, total = 3.5039 MB
  pv per batch = 0.0020 MB, total = 1.1680 MB
[Segment 2] Training Time = 3.44 sec
[Segment 2] GPU Memory Start = 356.30 MB
[Segment 2] GPU Memory End   = 356.89 MB
[Segment 2] GPU Peak Memory  = 357.94 MB

============= Done =============

=========== S

## 1E->1E

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


epo = 1


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal



base_cfg = Configs(
    seq_len          = 336 + pred_len//3,
    pred_len         = 192,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = f"./model2_ER_E{epo}_1E/",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

loss_fn = nn.MSELoss()



for seg in range(len(train_pred_segments_E1[0])):
    print(f"\n=========== Segment {seg} ===========")
    
    start_time = time.time()

    
    torch.cuda.reset_peak_memory_stats(device)
    mem_start = torch.cuda.memory_allocated(device)

    
    tr_pv = train_pred_segments_E1[:, seg, :, :]   
    va_pv = val_pred_segments_E1[:,  seg, :, :]

    tr_ds = TensorDataset(train_sequences, train_labels, tr_pv)
    va_ds = TensorDataset(val_sequences,   val_labels,   va_pv)
    tr_dl = DataLoader(tr_ds, batch_size=8, shuffle=True)
    va_dl = DataLoader(va_ds, batch_size=8, shuffle=False)

    
    model = Model(base_cfg).to(device)
    C     = model.channels                                    

    opts = []
    for c in range(C):
        params = list(model.Linear_Seasonal[c].parameters()) + \
                 list(model.Linear_Trend[c].parameters())
        opts.append(optim.Adam(params, lr=base_cfg.learning_rate))
        path = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}"
        path.mkdir(parents=True, exist_ok=True)
        
    active = set(range(C))            

    
    for epoch in range(1, 2):
        if not active: break          

        
        model.train()
        for c in active:                              
            if base_cfg.lradj=='type1':
                lr = base_cfg.learning_rate * 0.5**((epoch-1)//1)
                for g in opts[c].param_groups: g['lr'] = lr

        printed_tensor_info = False
        
        for xb, yb, pv in tr_dl:
            xb, yb, pv = xb.float().to(device), yb.float().to(device), pv.float().to(device)

            if not printed_tensor_info:
                
                xb_size = xb.numel() * xb.element_size() / 1024**2
                yb_size = yb.numel() * yb.element_size() / 1024**2
                pv_size = pv.numel() * pv.element_size() / 1024**2

                
                
                num_batches = len(tr_dl)

                
                total_xb_MB = num_batches * xb_size
                total_yb_MB = num_batches * yb_size
                total_pv_MB = num_batches * pv_size
                


                print(f"[Segment {seg}] Batch Size Info:")
                print(f"  xb per batch = {xb_size:.4f} MB, total = {total_xb_MB:.4f} MB")
                print(f"  yb per batch = {yb_size:.4f} MB, total = {total_yb_MB:.4f} MB")
                print(f"  pv per batch = {pv_size:.4f} MB, total = {total_pv_MB:.4f} MB")

                printed_tensor_info = True



            out = model(xb, pv)                       

            loss_list = []
            for c in active:
                pred = out[:, -base_cfg.pred_len:, c]
                tgt  = yb[:, -base_cfg.pred_len:, c]
                loss_list.append(loss_fn(pred, tgt))

            total = torch.stack(loss_list).sum()
            for opt in opts:
                if opt: opt.zero_grad()
            total.backward()
            for c in active:
                opts[c].step()

        
    elapsed = time.time() - start_time
    mem_end = torch.cuda.memory_allocated(device)
    peak_mem = torch.cuda.max_memory_allocated(device)


    print(f"[Segment {seg}] Training Time = {elapsed:.2f} sec")
    print(f"[Segment {seg}] GPU Memory Start = {mem_start/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Memory End   = {mem_end/1024**2:.2f} MB")
    print(f"[Segment {seg}] GPU Peak Memory  = {peak_mem/1024**2:.2f} MB")

    print(f"\n============= Done =============")

    
    torch.save(model.state_dict(),
                Path(base_cfg.save_path) / f"seg{seg}" / f"ch{c}" / "checkpoint.pth")
    del model, opts; torch.cuda.empty_cache()



=========== Segment 0 ===========
[Segment 0] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.1318 MB
  yb per batch = 0.0059 MB, total = 3.5039 MB
  pv per batch = 0.0020 MB, total = 1.1680 MB
[Segment 0] Training Time = 3.57 sec
[Segment 0] GPU Memory Start = 356.30 MB
[Segment 0] GPU Memory End   = 356.30 MB
[Segment 0] GPU Peak Memory  = 357.94 MB

============= Done =============

=========== Segment 1 ===========
[Segment 1] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.1318 MB
  yb per batch = 0.0059 MB, total = 3.5039 MB
  pv per batch = 0.0020 MB, total = 1.1680 MB
[Segment 1] Training Time = 2.50 sec
[Segment 1] GPU Memory Start = 356.30 MB
[Segment 1] GPU Memory End   = 356.89 MB
[Segment 1] GPU Peak Memory  = 357.94 MB

============= Done =============

=========== Segment 2 ===========
[Segment 2] Batch Size Info:
  xb per batch = 0.0103 MB, total = 6.1318 MB
  yb per batch = 0.0059 MB, total = 3.5039 MB
  pv per batch = 0.0020 MB, total = 1.1680 MB
[Segme

# 2nd Inference for Validation Set

In [36]:
train_segments_by_epoch = torch.load('./1st_results_ER/train_segments_by_epoch.pt', weights_only=False)
val_segments_by_epoch = torch.load('./1st_results_ER/val_segments_by_epoch.pt', weights_only=False)
test_segments_by_epoch = torch.load('./1st_results_ER/test_segments_by_epoch.pt', weights_only=False)

In [ ]:
train_pred_segments_ES = train_segments_by_epoch['ES']['train_pred_segments_EES']  
train_label_segments_ES = train_segments_by_epoch['ES']['train_label_segments_EES']
val_pred_segments_ES = val_segments_by_epoch['ES']['val_pred_segments_EES']  
val_label_segments_ES = val_segments_by_epoch['ES']['val_label_segments_EES']
test_pred_segments_ES = test_segments_by_epoch['ES']['test_pred_segments_EES']  
test_label_segments_ES = test_segments_by_epoch['ES']['test_label_segments_EES']


train_pred_segments_E1 = train_segments_by_epoch[1]['train_pred_segments_E1']  
train_label_segments_E1 = train_segments_by_epoch[1]['train_label_segments_E1']
val_pred_segments_E1 = val_segments_by_epoch[1]['val_pred_segments_E1']  
val_label_segments_E1 = val_segments_by_epoch[1]['val_label_segments_E1']
test_pred_segments_E1 = test_segments_by_epoch[1]['test_pred_segments_E1']  
test_label_segments_E1 = test_segments_by_epoch[1]['test_label_segments_E1']


## ES->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 192,
    seq_len          = 336 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_EES_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 65
batch_sz = 8

results2_ER_ES_val_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = val_pred_segments_ES[:, seg, :, :]         
    ds = TensorDataset(val_sequences, val_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()      
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ER_ES_val_1E[seg] = {
        "pred2_ES_val_1E"    : pred_all,   
        "true2_ES_val_1E"    : true_all,   
        "metrics2_ES_val_1E" : metrics     
    }


seg10_ch0_mse = results2_ER_ES_val_1E[10]["metrics2_ES_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.27551406621932983  MAE:0.4096173346042633
▶ seg0  mean  MSE:0.27551406621932983  MAE:0.4096173346042633

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.34079214930534363  MAE:0.4604310095310211
▶ seg1  mean  MSE:0.34079214930534363  MAE:0.4604310095310211

―――― Inference  |  Segment 2 ――――


  seg2-ch0  MSE:0.7591993808746338  MAE:0.7221771478652954
▶ seg2  mean  MSE:0.7591993808746338  MAE:0.7221771478652954

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.46572569012641907  MAE:0.5387759804725647
▶ seg3  mean  MSE:0.46572569012641907  MAE:0.5387759804725647

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.7928516864776611  MAE:0.7243279218673706
▶ seg4  mean  MSE:0.7928516864776611  MAE:0.7243279218673706

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.7522785067558289  MAE:0.7026906609535217
▶ seg5  mean  MSE:0.7522785067558289  MAE:0.7026906609535217

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.346533864736557  MAE:0.4590912461280823
▶ seg6  mean  MSE:0.346533864736557  MAE:0.4590912461280823

―――― Inference  |  Segment 7 ――――
  seg7-ch0  MSE:0.28308790922164917  MAE:0.4112105667591095
▶ seg7  mean  MSE:0.28308790922164917  MAE:0.4112105667591095

―――― Inference  |  Segment 8 ――――
  seg8-ch0  MSE:0.3696143627166748  MAE:0.4816242754459381
▶ seg8  m

## 1E->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 192,
    seq_len          = 336 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_E1_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path


C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 65
batch_sz = 8

results2_ER_E1_val_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = val_pred_segments_E1[:, seg, :, :]         
    ds = TensorDataset(val_sequences, val_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ER_E1_val_1E[seg] = {
        "pred2_E1_val_1E"    : pred_all,   
        "true2_E1_val_1E"    : true_all,   
        "metrics2_E1_val_1E" : metrics     
    }


seg10_ch0_mse = results2_ER_E1_val_1E[10]["metrics2_E1_val_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――


  seg0-ch0  MSE:0.4908599555492401  MAE:0.562467098236084
▶ seg0  mean  MSE:0.4908599555492401  MAE:0.562467098236084

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.5382745265960693  MAE:0.5776686668395996
▶ seg1  mean  MSE:0.5382745265960693  MAE:0.5776686668395996

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:1.0033131837844849  MAE:0.8410149216651917
▶ seg2  mean  MSE:1.0033131837844849  MAE:0.8410149216651917

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.4033108055591583  MAE:0.49783438444137573
▶ seg3  mean  MSE:0.4033108055591583  MAE:0.49783438444137573

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.5339670181274414  MAE:0.5806480646133423
▶ seg4  mean  MSE:0.5339670181274414  MAE:0.5806480646133423

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.36183038353919983  MAE:0.4734642505645752
▶ seg5  mean  MSE:0.36183038353919983  MAE:0.4734642505645752

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.3902229964733124  MAE:0.49480077624320984
▶ seg6  

# 2nd Inference for Test Set

## ES->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 192
seq_len = 336 + pred_len//3


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 192,
    seq_len          = 336 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_EES_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 65
batch_sz = 8

results2_ER_ES_test_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = test_pred_segments_ES[:, seg, :, :]         
    ds = TensorDataset(test_sequences, test_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ER_ES_test_1E[seg] = {
        "pred2_ES_test_1E"    : pred_all,   
        "true2_ES_test_1E"    : true_all,   
        "metrics2_ES_test_1E" : metrics     
    }


seg10_ch0_mse = results2_ER_ES_test_1E[10]["metrics2_ES_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.262395977973938  MAE:0.39186352491378784
▶ seg0  mean  MSE:0.262395977973938  MAE:0.39186352491378784

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.2221594899892807  MAE:0.3776102662086487
▶ seg1  mean  MSE:0.2221594899892807  MAE:0.3776102662086487

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.5651916861534119  MAE:0.614907443523407
▶ seg2  mean  MSE:0.5651916861534119  MAE:0.614907443523407

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.2550220489501953  MAE:0.387716144323349
▶ seg3  mean  MSE:0.2550220489501953  MAE:0.387716144323349

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.5454950928688049  MAE:0.5912182331085205
▶ seg4  mean  MSE:0.5454950928688049  MAE:0.5912182331085205

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.5049117207527161  MAE:0.5652082562446594
▶ seg5  mean  MSE:0.5049117207527161  MAE:0.5652082562446594

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.2201954871416092  MAE

## 1E->1E

### Configs

In [ ]:
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np, os
from pathlib import Path

pred_len = 192
seq_len = 336 + pred_len//3


class Configs:
    def __init__(self, seq_len, pred_len, enc_in, individual,
                 learning_rate, lradj, patience,
                 save_path, attach_to_trend, attach_to_seasonal):
        self.seq_len  = seq_len
        self.pred_len = pred_len
        self.enc_in   = enc_in          
        self.individual = individual
        self.learning_rate = learning_rate
        self.lradj = lradj
        self.patience = patience
        self.save_path = save_path
        self.attach_to_trend    = attach_to_trend
        self.attach_to_seasonal = attach_to_seasonal


base_cfg = Configs(
    pred_len         = 192,
    seq_len          = 336 + pred_len//3,
    enc_in           = 1,
    individual       = True,
    learning_rate    = 0.0005,
    lradj            = 'type1',
    patience         = 3,
    save_path        = "./model2_ER_E1_1E",
    attach_to_trend  = True,
    attach_to_seasonal = True
)
Path(base_cfg.save_path).mkdir(parents=True, exist_ok=True)

### Inference

In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from pathlib import Path

C        = 1                                
P        = base_cfg.pred_len                
seg_cnt  = 65
batch_sz = 8

results2_ER_E1_test_1E = {}  

for seg in range(seg_cnt):
    print(f"\n―――― Inference  |  Segment {seg} ――――")

    
    pv = test_pred_segments_E1[:, seg, :, :]         
    ds = TensorDataset(test_sequences, test_labels, pv) 
    dl = DataLoader(ds, batch_size=batch_sz, shuffle=False)

    
    pred_ch, true_ch, metrics = [], [], {}

    
    for ch in range(C):
        ckpt = Path(base_cfg.save_path) / f"seg{seg}" / f"ch{ch}" / "checkpoint.pth"
        if not ckpt.exists():
            print(f"[Warn] seg{seg}-ch{ch} ckpt 없음 → skip");  continue

        
        model = Model(base_cfg).to(device)
        model.load_state_dict(torch.load(ckpt, map_location=device))
        model.eval()

        preds, trues = [], []
        with torch.no_grad():
            for xb, yb, pv_batch in dl:                 
                xb  = xb.float().to(device)
                yb  = yb.float()        
                pv_ = pv_batch.float().to(device)

                out = model(xb, pv_)                     
                preds.append(out[:, -P:, ch].cpu())
                trues.append(yb[:, -P:, ch])

        preds = torch.cat(preds, 0)     
        trues = torch.cat(trues, 0)
        pred_ch.append(preds)
        true_ch.append(trues)

        
        p, t = preds.reshape(-1).cpu().numpy(), trues.reshape(-1).cpu().numpy()
        mse  = np.mean((p - t)**2)
        mae  = np.mean(np.abs(p - t))
        metrics[ch] = (mse, mae)

        print(f"  seg{seg}-ch{ch}  MSE:{mse}  MAE:{mae}")

        del model; torch.cuda.empty_cache()

    
    pred_all = torch.stack(pred_ch,  dim=2)   
    true_all = torch.stack(true_ch,  dim=2)

    
    mse_avg = np.mean([m for m,_ in metrics.values()])
    mae_avg = np.mean([a for _,a in metrics.values()])
    print(f"▶ seg{seg}  mean  MSE:{mse_avg}  MAE:{mae_avg}")

    
    results2_ER_E1_test_1E[seg] = {
        "pred2_E1_test_1E"    : pred_all,   
        "true2_E1_test_1E"    : true_all,   
        "metrics2_E1_test_1E" : metrics     
    }


seg10_ch0_mse = results2_ER_E1_test_1E[10]["metrics2_E1_test_1E"][0][0]
print("\nseg10 ch0 MSE :", seg10_ch0_mse)



―――― Inference  |  Segment 0 ――――
  seg0-ch0  MSE:0.2614332437515259  MAE:0.39819425344467163
▶ seg0  mean  MSE:0.2614332437515259  MAE:0.39819425344467163

―――― Inference  |  Segment 1 ――――
  seg1-ch0  MSE:0.3571838438510895  MAE:0.45173004269599915
▶ seg1  mean  MSE:0.3571838438510895  MAE:0.45173004269599915

―――― Inference  |  Segment 2 ――――
  seg2-ch0  MSE:0.7086803317070007  MAE:0.6865984201431274
▶ seg2  mean  MSE:0.7086803317070007  MAE:0.6865984201431274

―――― Inference  |  Segment 3 ――――
  seg3-ch0  MSE:0.25056150555610657  MAE:0.3922785818576813
▶ seg3  mean  MSE:0.25056150555610657  MAE:0.3922785818576813

―――― Inference  |  Segment 4 ――――
  seg4-ch0  MSE:0.3265547752380371  MAE:0.44387322664260864
▶ seg4  mean  MSE:0.3265547752380371  MAE:0.44387322664260864

―――― Inference  |  Segment 5 ――――
  seg5-ch0  MSE:0.20299610495567322  MAE:0.3549615740776062
▶ seg5  mean  MSE:0.20299610495567322  MAE:0.3549615740776062

―――― Inference  |  Segment 6 ――――
  seg6-ch0  MSE:0.2164320

# Rank Models (by 2nd alidation performance)

In [46]:
results1_ER_train = torch.load("./1st_results_ER/train_1st_results.pt")
results1_ER_val = torch.load("./1st_results_ER/val_1st_results.pt")
results1_ER_test = torch.load("./1st_results_ER/test_1st_results.pt")

### ES->1E

In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error




TOP_N       = 65
SEG_CNT     = 65
CH_CNT      = 1
metric_idx  = 0          




mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ER_ES_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ER_ES_val_1E[seg]["metrics2_ES_val_1E"]:
            mse_table[seg, ch] = results2_ER_ES_val_1E[seg]["metrics2_ES_val_1E"][ch][metric_idx]




print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_ES_val_1E = {}                       

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ER_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    
    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    
    top_seg_idx_ES_val_1E[ch] = [seg for seg, _ in top]   

    
    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()




print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_ES_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.356182

  Top  1 | seg 59 | MSE 0.252418 (+0.103764)
  Top  2 | seg 30 | MSE 0.260451 (+0.095731)
  Top  3 | seg 64 | MSE 0.267510 (+0.088672)
  Top  4 | seg 54 | MSE 0.271194 (+0.084988)
  Top  5 | seg 38 | MSE 0.272306 (+0.083876)
  Top  6 | seg 27 | MSE 0.273943 (+0.082239)
  Top  7 | seg 00 | MSE 0.275514 (+0.080668)
  Top  8 | seg 56 | MSE 0.280015 (+0.076167)
  Top  9 | seg 53 | MSE 0.281273 (+0.074909)
  Top 10 | seg 58 | MSE 0.282030 (+0.074152)
  Top 11 | seg 28 | MSE 0.282031 (+0.074151)
  Top 12 | seg 07 | MSE 0.283088 (+0.073094)
  Top 13 | seg 21 | MSE 0.289368 (+0.066814)
  Top 14 | seg 49 | MSE 0.289394 (+0.066788)
  Top 15 | seg 11 | MSE 0.294675 (+0.061507)
  Top 16 | seg 62 | MSE 0.295256 (+0.060926)
  Top 17 | seg 46 | MSE 0.297099 (+0.059083)
  Top 18 | seg 44 | MSE 0.301542 (+0.054640)
  Top 19 | seg 33 | MSE 0.305755 (+0.050427)
  Top 20 | seg 12 | MSE 0.306008 (+0.050174)
  Top 21 | seg 47 |

### 1E->1E

In [ ]:

import numpy as np
from sklearn.metrics import mean_squared_error




TOP_N       = 65
SEG_CNT     = 65
CH_CNT      = 1
metric_idx  = 0          




mse_table = np.full((SEG_CNT, CH_CNT), np.inf)

for seg in range(SEG_CNT):
    if seg not in results2_ER_E1_val_1E:
        continue
    for ch in range(CH_CNT):
        if ch in results2_ER_E1_val_1E[seg]["metrics2_E1_val_1E"]:
            mse_table[seg, ch] = results2_ER_E1_val_1E[seg]["metrics2_E1_val_1E"][ch][metric_idx]




print("\n▶ Train-set  Per-Channel  Top-N  MSE\n")
top_seg_idx_E1_val_1E = {}                       

for ch in range(CH_CNT):
    print(f"=== Channel {ch} ===")
    v_mse = results1_ER_val[1]['mse_val_E1']
    print(f"Vanilla MSE : {v_mse:.6f}\n")

    
    pairs = [(seg, mse_table[seg, ch]) for seg in range(SEG_CNT)
             if np.isfinite(mse_table[seg, ch])]
    top   = sorted(pairs, key=lambda x: x[1])[:TOP_N]

    
    top_seg_idx_E1_val_1E[ch] = [seg for seg, _ in top]   

    
    for rank, (seg_id, mse_val) in enumerate(top, 1):
        delta = v_mse - mse_val
        sign  = "+" if delta > 0 else "-"
        print(f"  Top {rank:2d} | seg {seg_id:02d} | MSE {mse_val:.6f} "
              f"({sign}{abs(delta):.6f})")
    print()




print("Saved top segment indices per channel:")
for ch, seg_list in top_seg_idx_E1_val_1E.items():
    print(f"Channel {ch}: {seg_list}")



▶ Train-set  Per-Channel  Top-N  MSE

=== Channel 0 ===
Vanilla MSE : 0.356182

  Top  1 | seg 20 | MSE 0.261713 (+0.094469)
  Top  2 | seg 63 | MSE 0.264884 (+0.091298)
  Top  3 | seg 14 | MSE 0.265787 (+0.090395)
  Top  4 | seg 41 | MSE 0.274182 (+0.082000)
  Top  5 | seg 31 | MSE 0.277994 (+0.078188)
  Top  6 | seg 34 | MSE 0.278258 (+0.077924)
  Top  7 | seg 10 | MSE 0.282703 (+0.073479)
  Top  8 | seg 40 | MSE 0.284201 (+0.071981)
  Top  9 | seg 35 | MSE 0.291845 (+0.064337)
  Top 10 | seg 09 | MSE 0.292855 (+0.063327)
  Top 11 | seg 64 | MSE 0.294940 (+0.061242)
  Top 12 | seg 47 | MSE 0.297242 (+0.058940)
  Top 13 | seg 59 | MSE 0.297450 (+0.058732)
  Top 14 | seg 08 | MSE 0.299347 (+0.056835)
  Top 15 | seg 28 | MSE 0.305863 (+0.050319)
  Top 16 | seg 48 | MSE 0.319334 (+0.036848)
  Top 17 | seg 56 | MSE 0.320191 (+0.035991)
  Top 18 | seg 58 | MSE 0.322039 (+0.034143)
  Top 19 | seg 24 | MSE 0.330672 (+0.025510)
  Top 20 | seg 21 | MSE 0.344441 (+0.011741)
  Top 21 | seg 36 |

# Ensemble

## ES->1E


In [ ]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 65
CH_CNT   = 1
batch_sz = 8

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   
    return torch.cat(buf, 0)   


van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []




for ch in range(CH_CNT):
    print(f"\n=== Channel {ch} ===")

    
    
    
    v_mse = results1_ER_test['ES']['mse_test_EES']
    v_mae = results1_ER_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    
    for seg in top_seg_idx_ES_val_1E[ch][:TOP_K]:
        ckpt = f"./model2_ER_EES_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] ckpt missing : seg{seg}")
            continue

        pv_seg = test_pred_segments_ES[:, seg, :, :]
        dl     = DataLoader(TensorDataset(test_sequences, test_labels, pv_seg),
                            batch_size=batch_sz, shuffle=False)

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        preds_test_arr.append(infer_one(mdl, dl, ch))   
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    
    
    
    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list, rho_list = [], []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)

        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min)/(V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min)/(R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1
    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]

    best_K = Ks[int(np.argmin(scores))]

    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    
    
    
    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    
    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    
    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)

    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    print(f"[Var-Corr]  K={best_K}")
    print(f"  MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"  MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")




if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE     : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%) : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE     : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%) : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.307999 | MAE : 0.406738
  — Var / |Corr| / Score —
   K= 5 | Var=3.379e-02 | |ρ|=0.9825 | S=1.1157 
   K=10 | Var=2.962e-02 | |ρ|=0.9799 | S=0.9571 
   K=15 | Var=2.789e-02 | |ρ|=0.9778 | S=0.8593 
   K=20 | Var=3.245e-02 | |ρ|=0.9741 | S=0.8373 <- pick
   K=25 | Var=3.471e-02 | |ρ|=0.9751 | S=0.9130 
   K=30 | Var=3.562e-02 | |ρ|=0.9728 | S=0.8610 
   K=35 | Var=3.839e-02 | |ρ|=0.9708 | S=0.8558 
   K=40 | Var=4.143e-02 | |ρ|=0.9698 | S=0.8850 
   K=45 | Var=4.620e-02 | |ρ|=0.9686 | S=0.9431 
   K=50 | Var=4.949e-02 | |ρ|=0.9657 | S=0.9203 
   K=55 | Var=5.636e-02 | |ρ|=0.9606 | S=0.9041 
   K=60 | Var=6.397e-02 | |ρ|=0.9556 | S=0.9029 
   K=65 | Var=7.889e-02 | |ρ|=0.9490 | S=1.0000 
[Var-Corr]  K=20
  MSE = 0.215891 | Δ = +0.092108 | Δ% = +29.91%
  MAE = 0.355194 | Δ = +0.051544 | Δ% = +12.67%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE : 0.307999
Ensemble mean MSE : 0.215891
Average Δ MSE     : +0.092108
Average Δ MSE (%) : +29.91%


## 1E->1E


In [50]:
results1_ER_test['ES']['mae_test_EES']

0.4067384600639343

In [ ]:
import torch, os, numpy as np
from torch.utils.data import DataLoader, TensorDataset

TOP_K    = 65
CH_CNT   = 1
batch_sz = 8

def infer_one(model, loader, ch):
    buf = []
    model.eval()
    with torch.no_grad():
        for xb, yb, pv in loader:
            xb, pv = xb.to(device), pv.to(device)
            buf.append(model(xb, pv)[:, :, ch].cpu())   
    return torch.cat(buf, 0)    



van_mse_list, ens_mse_list, delta_mse_list, delta_mse_pct_list = [], [], [], []
van_mae_list, ens_mae_list, delta_mae_list, delta_mae_pct_list = [], [], [], []





for ch in range(CH_CNT):

    print(f"\n=== Channel {ch} ===")

    
    v_mse = results1_ER_test['ES']['mse_test_EES']
    v_mae = results1_ER_test['ES']['mae_test_EES']

    print(f"[Vanilla]   MSE : {v_mse:.6f} | MAE : {v_mae:.6f}")

    van_mse_list.append(v_mse)
    van_mae_list.append(v_mae)

    preds_test_arr = []
    ok_segs = []

    
    
    
    for seg in top_seg_idx_E1_val_1E[ch][:TOP_K]:

        ckpt = f"./model2_ER_E1_1E/seg{seg}/ch{ch}/checkpoint.pth"
        if not os.path.exists(ckpt):
            print(f"  [skip] missing checkpoint : seg{seg}")
            continue

        mdl = Model(base_cfg).to(device)
        mdl.load_state_dict(torch.load(ckpt, map_location=device))

        pv_seg = test_pred_segments_E1[:, seg, :, :]
        dl = DataLoader(
            TensorDataset(test_sequences, test_labels, pv_seg),
            batch_size=batch_sz, shuffle=False
        )

        preds_test_arr.append(infer_one(mdl, dl, ch))
        ok_segs.append(seg)

    if len(preds_test_arr) < 5:
        print("  Not enough valid models.")
        continue

    
    
    
    step = 5
    Ks = list(range(step, len(preds_test_arr)+1, step))

    var_list = []
    rho_list = []

    for K in Ks:
        F = torch.stack(preds_test_arr[:K], dim=0)   

        
        V = torch.var(F, dim=0).mean().item()
        var_list.append(V)

        
        flat = F.reshape(K, -1).numpy()
        corr = np.corrcoef(flat)
        R    = np.mean(np.abs(corr[np.triu_indices(K, 1)]))
        rho_list.append(R)

    
    
    
    V_min, V_max = min(var_list), max(var_list)
    R_min, R_max = min(rho_list), max(rho_list)

    V_norm = [(v - V_min) / (V_max - V_min + 1e-12) for v in var_list]
    R_norm = [(r - R_min) / (R_max - R_min + 1e-12) for r in rho_list]

    alpha, beta = 1, 1.35   

    scores = [alpha*v + beta*r for v, r in zip(V_norm, R_norm)]
    best_K = Ks[int(np.argmin(scores))]

    
    
    
    print("  — Var / |Corr| / Score —")
    for k, v, r, s in zip(Ks, var_list, rho_list, scores):
        tag = "<- pick" if k == best_K else ""
        print(f"   K={k:2d} | Var={v:.3e} | |ρ|={r:.4f} | S={s:.4f} {tag}")

    
    
    
    ens_pred = torch.mean(torch.stack(preds_test_arr[:best_K]), dim=0)  
    y_true   = test_labels[:, -base_cfg.pred_len:, ch].cpu()

    mse_ens = torch.mean((ens_pred - y_true)**2).item()
    mae_ens = torch.mean(torch.abs(ens_pred - y_true)).item()

    ens_mse_list.append(mse_ens)
    ens_mae_list.append(mae_ens)

    
    delta_mse = v_mse - mse_ens
    delta_mae = v_mae - mae_ens

    
    delta_mse_pct = (delta_mse / v_mse) * 100
    delta_mae_pct = (delta_mae / v_mae) * 100

    delta_mse_list.append(delta_mse)
    delta_mae_list.append(delta_mae)
    delta_mse_pct_list.append(delta_mse_pct)
    delta_mae_pct_list.append(delta_mae_pct)

    sign = "+" if delta_mse > 0 else "-"
    print(f"[Var-Corr]  K={best_K}")
    print(f"   MSE = {mse_ens:.6f} | Δ = {delta_mse:+.6f} | Δ% = {delta_mse_pct:+.2f}%")
    print(f"   MAE = {mae_ens:.6f} | Δ = {delta_mae:+.6f} | Δ% = {delta_mae_pct:+.2f}%")





if ens_mse_list:
    print("\n──────── Summary (Var-Corr) ───────")
    print(f"Vanilla  mean MSE  : {np.mean(van_mse_list):.6f}")
    print(f"Ensemble mean MSE  : {np.mean(ens_mse_list):.6f}")
    print(f"Average Δ MSE      : {np.mean(delta_mse_list):+.6f}")
    print(f"Average Δ MSE (%)  : {np.mean(delta_mse_pct_list):+.2f}%\n")

    print(f"Vanilla  mean MAE  : {np.mean(van_mae_list):.6f}")
    print(f"Ensemble mean MAE  : {np.mean(ens_mae_list):.6f}")
    print(f"Average Δ MAE      : {np.mean(delta_mae_list):+.6f}")
    print(f"Average Δ MAE (%)  : {np.mean(delta_mae_pct_list):+.2f}%")



=== Channel 0 ===
[Vanilla]   MSE : 0.307999 | MAE : 0.406738
  — Var / |Corr| / Score —
   K= 5 | Var=1.736e-02 | |ρ|=0.9843 | S=1.3500 
   K=10 | Var=1.939e-02 | |ρ|=0.9776 | S=1.1520 
   K=15 | Var=2.339e-02 | |ρ|=0.9730 | S=1.0687 
   K=20 | Var=2.594e-02 | |ρ|=0.9717 | S=1.0759 
   K=25 | Var=3.014e-02 | |ρ|=0.9690 | S=1.0711 
   K=30 | Var=3.178e-02 | |ρ|=0.9695 | S=1.1222 
   K=35 | Var=3.454e-02 | |ρ|=0.9675 | S=1.1106 
   K=40 | Var=4.092e-02 | |ρ|=0.9634 | S=1.0996 
   K=45 | Var=4.362e-02 | |ρ|=0.9606 | S=1.0566 
   K=50 | Var=4.599e-02 | |ρ|=0.9582 | S=1.0193 
   K=55 | Var=4.787e-02 | |ρ|=0.9549 | S=0.9393 <- pick
   K=60 | Var=5.269e-02 | |ρ|=0.9522 | S=0.9465 
   K=65 | Var=6.331e-02 | |ρ|=0.9473 | S=1.0000 
[Var-Corr]  K=55
   MSE = 0.200147 | Δ = +0.107852 | Δ% = +35.02%
   MAE = 0.353517 | Δ = +0.053221 | Δ% = +13.08%

──────── Summary (Var-Corr) ───────
Vanilla  mean MSE  : 0.307999
Ensemble mean MSE  : 0.200147
Average Δ MSE      : +0.107852
Average Δ MSE (%)  : +3